## **Machine learning Project**
### By **Thomas Marchal** #20220714, **Louis Hislaire** #  and **Jules Rougier** # 


Step 1:
We choose the Dataset number D : House_data


Step 2: Scenario / About Dataset -> Jules

Step 3: Data Loading -> Thomas


In [ ]:
# Load required librairies
import numpy as np
import pandas as pd

In [ ]:
#Load the data
df = pd.read_csv("data/House_data.csv")
df.head()

Step 4: Data Wrangling or Data Pre-processing -> Louis

Step 5: Exploratory Data Analysis -> Thomas

Step 6: Model Developement (Apply ML algorithms)
BONUS if we choose to use 2ML algorithms (see pdf) 1 normal + 2 dans les Bonus
Thomas/Louis/Jules

Step 7: Model Evaluation, Implement and compare different metrics to evaluate the quality of the model.
Chaque model Thomas/Louis/Jules

Step 8: Model Refinement -> Jules

Harmoniser le doc pour le rendu